# Analyse des sentiments pour la  finance

L'analyse du sentiment financier est utilisée pour extraire des informations à partir des actualités, des médias sociaux, des rapports financiers et des données alternatives pour l'investissement, le trading, la gestion des risques, les opérations dans les institutions financières et essentiellement tout ce qui concerne la finance. Dans de nombreux cas, il est devenu inefficace car de nombreux acteurs du marché le comprennent et ont amélioré cette technique.


Nous voulons prédire le cours des actions avec les titres des actualités et des articles.
On a récupérer des articles sur nos 3 entreprises sur la même période que les données sur les prix.

Voici les étapes pour exécuter notre projet d'analyse des sentiments:

- Rassembler les titres et les dates des articles

- Importer et nettoyer les données (traitement de texte)

- Exécutez une analyse des sentiments et créez un index de score

- Corréler l'indice de score décalé avec les prix


In [1]:
import pandas as pd
import numpy as np
import re
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
#from textblob import TexBlob
from urllib.request import Request, urlopen
import json
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import classification_report
import requests
from bs4 import BeautifulSoup

## Rassembler les titres des articles

In [2]:
df_mzn = pd.read_csv('amazon_News2.csv')
df_goog = pd.read_csv('google_News2.csv')
df_msft = pd.read_csv('microsoft_News2.csv')

In [3]:
df_msft.head()

,Unnamed: 0,Date,Title
0,0,2020-10-01,Datadog price target lifted on Microsoft partn...
1,1,2020-09-21,Analog Devices teams with Microsoft on 3D prod...
2,2,2020-09-08,"Microsoft Teams, NFL deepen gameday partnership"
3,3,2020-09-05,Microsoft teams up with BP to achieve net zero...
4,4,2020-09-11,Microsoft targets raised ahead of earnings


In [4]:
# Suppression de la première colonne
df_mzn = df_mzn.drop("Unnamed: 0", axis = 1)
df_goog = df_goog.drop("Unnamed: 0", axis = 1)
df_msft = df_msft.drop("Unnamed: 0", axis = 1)

In [5]:
df_msft.head()

,Date,Title
0,2020-10-01,Datadog price target lifted on Microsoft partn...
1,2020-09-21,Analog Devices teams with Microsoft on 3D prod...
2,2020-09-08,"Microsoft Teams, NFL deepen gameday partnership"
3,2020-09-05,Microsoft teams up with BP to achieve net zero...
4,2020-09-11,Microsoft targets raised ahead of earnings


## Exécutez une analyse des sentiments et créez un index de score

La partie suivante consiste à envoyer nos titres dans un analyseur de sentiment pour générer un score.

VADER est un analyseur de sentiment qui est formé à l'aide des médias sociaux et des données d'actualité en utilisant une approche basée sur le lexique. Cela signifie qu'il examine les mots, la ponctuation, les phases, les émojis, etc. et les classe comme positifs ou négatifs.

VADER signifie «Valence Aware Dictionary and sEntiment Reasoner».

In [6]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\ouizb\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [7]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

results = []

for headline in df_msft['Title']:
    pol_score = SIA().polarity_scores(headline) # run analysis
    pol_score['headline'] = headline # add headlines for viewing
    results.append(pol_score)

results[0]

{'neg': 0.0,
 'neu': 1.0,
 'pos': 0.0,
 'compound': 0.0,
 'headline': 'Datadog price target lifted on Microsoft partnership tailwind'}

Ensuite, nous concaténons cette liste à notre dataframe d'origine. Cependant, nous ne nous intéressons qu'aux valeurs de la variable «compound».

In [8]:
df_msft['Score'] = pd.DataFrame(results)['compound']

In [9]:
df_msft

,Date,Title,Score
0,2020-10-01,Datadog price target lifted on Microsoft partn...,0.0000
1,2020-09-21,Analog Devices teams with Microsoft on 3D prod...,0.0000
2,2020-09-08,"Microsoft Teams, NFL deepen gameday partnership",0.0000
3,2020-09-05,Microsoft teams up with BP to achieve net zero...,0.0000
4,2020-09-11,Microsoft targets raised ahead of earnings,0.0000
...,...,...,...
330,2016-10-03,Office iPad apps arriving ... eventually,0.0000
331,2014-06-19,"IBM, Cisco make OpenStack cloud acquisitions",0.0000
332,2016-03-18,NBC Sports launches dedicated video-streaming ...,0.4588
333,2013-12-12,Barracuda down 23.8% after billings guidance c...,-0.2732


In [10]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

results = []

for headline in df_mzn['Title']:
    pol_score = SIA().polarity_scores(headline) # run analysis
    pol_score['headline'] = headline # add headlines for viewing
    results.append(pol_score)

results[0]

{'neg': 0.0,
 'neu': 0.866,
 'pos': 0.134,
 'compound': 0.1779,
 'headline': 'Amazon Music partners with Universal Music, Warner Music on Ultra HD remasters'}

In [11]:
df_mzn['Score'] = pd.DataFrame(results)['compound']

In [12]:
df_mzn

,Date,Title,Score
0,2020-10-01,"Amazon Music partners with Universal Music, Wa...",0.1779
1,2020-09-22,"Amazon to expand tech hubs, corporate offices,...",0.4588
2,2020-09-22,"Amazon hires former Yahoo, Slack privacy head ...",0.1779
3,2020-09-16,"Amazon's latest job spree: Another 100,000 wor...",0.0000
4,2020-08-26,Amazon targeted by German antitrust probe,0.1779
...,...,...,...
475,2018-11-05,"S&P, Dow post gains while Nasdaq slips",0.3400
476,2017-08-28,Morgan Stanley pulls bearish outlook for Roku;...,0.2960
477,2018-10-25,Premarket Losers as of 9:05 am (06/28/2018),-0.5267
478,2018-06-21,Piper's fall survey: Instagram beats Snap in t...,0.0000


In [13]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

results = []

for headline in df_goog['Title']:
    pol_score = SIA().polarity_scores(headline) # run analysis
    pol_score['headline'] = headline # add headlines for viewing
    results.append(pol_score)

results[0]

{'neg': 0.0,
 'neu': 1.0,
 'pos': 0.0,
 'compound': 0.0,
 'headline': 'Google pauses News Showcase launch in Australia over royalties spat'}

In [14]:
df_goog['Score'] = pd.DataFrame(results)['compound']

In [15]:
df_goog

,Date,Title,Score
0,2020-10-01,Google pauses News Showcase launch in Australi...,0.0000
1,2020-09-25,Microsoft's Bing wins slots in Google's EU mob...,0.5719
2,2020-08-26,"Italy probes Apple, Google and Dropbox cloud s...",0.0000
3,2020-08-21,"Facebook, Google withdraw FCC request for unde...",0.0000
4,2020-08-04,"Samsung to make chips for Cisco, Google - report",0.0000
...,...,...,...
364,2011-12-28,Google (NASDAQ:GOOG) is having a hard time fin...,-0.1027
365,2008-06-04,Enterprise ogre: Oracle (NYSE:ORCL) buys Sales...,0.0000
366,2008-06-03,"Google (NASDAQ:GOOG), Microsoft (NASDAQ:MSFT),...",0.0000
367,2018-07-13,YouTube TV expands virtually nationwide,0.1027


In [16]:
df_goog.Date.unique().shape

(349,)

##  Corréler l'indice de score décalé avec les prix

 Nous voulons comparer la relation entre les rendements des actions et notre score de sentiment. S'il existe une relation significative, nos scores de sentiment peuvent avoir une valeur prédictive.

### Agréger les scores de sentiment quotidiens

Nous n'avons besoin que d'un seul score par jour pour comparer les prix quotidiens.

In [17]:
df_mzn = df_mzn.groupby(['Date']).sum()
df_goog = df_goog.groupby(['Date']).sum()
df_msft = df_msft.groupby(['Date']).sum()

In [18]:
df_msft

,Score
Date,
2008-06-12,0.000
2008-06-18,0.000
2008-06-26,0.000
2008-07-02,0.000
2008-07-13,0.000
...,...
2020-09-18,-0.128
2020-09-21,0.000
2020-09-24,0.000


### Importez les prix  des entreprises et calculez les retours
L'objectif de cette étape est d'obtenir les rendements quotidiens (et non les cours des actions)

In [19]:
amzn_p = pd.read_csv("amazon2.csv")
goog_p = pd.read_csv("google2.csv")
msft_p = pd.read_csv("microsoft2.csv")

In [20]:
amzn_p.head()

,Unnamed: 0,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Mean_Price
0,0,2010-09-30,160.01,160.93,155.60,157.06,7606000,0,0,158.265
1,1,2010-10-01,157.08,157.44,152.20,153.71,8683400,0,0,154.820
2,2,2010-10-04,153.95,155.96,152.79,155.39,5346500,0,0,154.375
3,3,2010-10-05,157.08,161.21,157.01,160.87,5620000,0,0,159.110
4,4,2010-10-06,160.60,160.68,154.60,155.40,6060800,0,0,157.640


In [21]:
# Je garde juste les colonnes dont j'ai besoin
amzn= amzn_p[['Date','Close']]
goog= goog_p[['Date','Close']]
msft= msft_p[['Date','Close']]

In [22]:
amzn.head()

,Date,Close
0,2010-09-30,157.06
1,2010-10-01,153.71
2,2010-10-04,155.39
3,2010-10-05,160.87
4,2010-10-06,155.40


Maintenant que nous avons nos prix, nous devons calculer nos rendements.
Pour calculer les rendements quotidiens, nous divisons les prix d'aujourd'hui par ceux d'hier.

In [23]:
amzn['Returns'] = amzn['Close']/amzn['Close'].shift(1) - 1 # calculate daily returns
goog['Returns'] = goog['Close']/goog['Close'].shift(1) - 1
msft['Returns'] = msft['Close']/msft['Close'].shift(1) - 1

C:\Users\ouizb\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\ouizb\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\ouizb\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

In [24]:
msft.head()

,Date,Close,Returns
0,2010-09-30,19.40,NaN
1,2010-10-01,19.32,-0.004124
2,2010-10-04,18.94,-0.019669
3,2010-10-05,19.29,0.018479
4,2010-10-06,19.36,0.003629


### Vérifiez la relation entre le score retardé et les rendements (quotidiennement)
L'objectif de cette étape est de vérifier si le score de sentiment prédit les rendements futurs des actions.
Un score de sentiment décalé d'un jour nous permet de comparer les titres des articles d'aujourd'hui aux rendements boursiers de demain.

#### Décalage du score de sentiment

In [25]:
# Ce code décale toutes les données d'une ligne vers le bas.
df_msft['Score(1)'] = df_msft['Score'].shift(1)
df_goog['Score(1)'] = df_goog['Score'].shift(1)
df_mzn['Score(1)'] = df_mzn['Score'].shift(1)

In [26]:
df_goog.head()

,Score,Score(1)
Date,,
2008-06-03,0.0000,NaN
2008-06-04,0.0000,0.0000
2008-07-03,0.0000,0.0000
2008-08-15,0.4404,0.0000
2008-09-02,0.0000,0.4404


In [27]:
df_goog.shape[0]

349

In [28]:
goog.shape[0]

2519

Le nombre de lignes de notre indice de score n'est pas le même que le nombre de lignes de nos retours.
Cela se produit car il y a des jours de bourse où il n'y a pas de nouvelles.

Nous devons faire correspondre les rendements quotidiens aux scores de sentiment correspondants

In [29]:
# je mets la date en index
amzn.index = amzn.Date
goog.index = goog.Date
msft.index = msft.Date

In [30]:
amzn = amzn.drop("Date", axis = 1)
goog = goog.drop("Date", axis = 1)
msft = msft.drop("Date", axis = 1)

In [31]:
msft.head()

,Close,Returns
Date,,
2010-09-30,19.40,NaN
2010-10-01,19.32,-0.004124
2010-10-04,18.94,-0.019669
2010-10-05,19.29,0.018479
2010-10-06,19.36,0.003629


Le code ci-dessus créera une nouvelle dateframe qui utilise les retours des actions comme référence et extraira le score de sentiment retardé approprié.

In [32]:
df3_amzn = pd.merge(amzn[['Returns']], df_mzn[['Score(1)']], left_index=True, right_index=True, how='left')
df3_goog = pd.merge(goog[['Returns']], df_goog[['Score(1)']], left_index=True, right_index=True, how='left')
df3_msft = pd.merge(msft[['Returns']], df_msft[['Score(1)']], left_index=True, right_index=True, how='left')

In [33]:
df3_goog.head()

,Returns,Score(1)
Date,,
2010-09-30,NaN,NaN
2010-10-01,-0.000305,NaN
2010-10-04,-0.006225,NaN
2010-10-05,0.030400,NaN
2010-10-06,-0.007199,NaN


In [34]:
# On va remplaçer tous les NaN par 0.
df3_goog.fillna(0, inplace=True) 
df3_amzn.fillna(0, inplace=True)
df3_msft.fillna(0, inplace=True)

In [35]:
df3_goog.head()

,Returns,Score(1)
Date,,
2010-09-30,0.000000,0.0
2010-10-01,-0.000305,0.0
2010-10-04,-0.006225,0.0
2010-10-05,0.030400,0.0
2010-10-06,-0.007199,0.0


#### Concevoir le test
 Le test consiste à vérifier la relation entre les scores de sentiment quotidiens et les rendements quotidiens.

In [36]:
# Nous supposerons qu'un score compris entre -0,5 et 0,5 est insignifiant pour des
df4_goog = df3_goog[(df3_goog['Score(1)'] > 0.5) | (df3_goog['Score(1)'] < -0.5)]
df4_amzn = df3_amzn[(df3_amzn['Score(1)'] > 0.5) | (df3_amzn['Score(1)'] < -0.5)]
df4_msft = df3_msft[(df3_msft['Score(1)'] > 0.5) | (df3_msft['Score(1)'] < -0.5)]

Nous avons défini notre hypothèse comme telle:

Un score de sentiment> 0,5 ou <-0,5 a une valeur prédictive uniquement sur les rendements quotidiens des actions  de demain.

In [37]:
df4_msft.head()

,Returns,Score(1)
Date,,
2011-07-12,-0.003257,0.5574
2012-04-27,-0.004159,0.5994
2012-06-20,0.007466,0.6597
2016-07-19,-0.016177,0.7456
2017-06-28,0.008507,-0.6124


#### Testez la valeur prédictive
Nous devons maintenant tester s'il existe une relation positive entre le score de sentiment retardé et les rendements quotidien.
Exécutons une analyse de corrélation.

In [38]:
msft_corr = df4_msft['Returns'].corr(df4_msft['Score(1)'])
amzn_corr = df4_amzn['Returns'].corr(df4_amzn['Score(1)'])
goog_corr = df4_goog['Returns'].corr(df4_goog['Score(1)'])

In [40]:
print('La corrélation entre le score des sentiments retardés et le rendement quotidien')
print(f'est de {goog_corr} pour Google')  
print(f'est de {amzn_corr} pour Amazon') 
print(f'est de {msft_corr} pour Microsoft') 

La corrélation entre le score des sentiments retardés et le rendement quotidien
est de 0.15557503263738798 pour Google
est de -0.037706647552581476 pour Amazon
est de -0.29350576549047797 pour Microsoft


#### Évaluations des résultats

Les coefficients de corrélation sont assez proche de 0. Cela signifie que les titres des articles seuls n'ont aucune valeur prédictive pour les rendements boursiers du lendemain.
Pas de surprise ici. Les marchés sont de plus en plus sophistiqués.